# NEO Detection Simulation

In [1]:
# Standard libraries
import datetime
import math
import pathlib
import sqlite3
import sys

# Installed libraries
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import spiceypy
import tqdm

# Append to root directory of this repository
sys.path.append("../")

# Auxiliary module that contains the apparent magnitude
from auxiliary import photometry

In [2]:
# Accessing the NEO database
database_dir = pathlib.Path("../databases/neos/")
database_file = pathlib.Path("neodys.db")
database_filepath = database_dir / database_file

# Establish a connection to the database and set a cursor
neodys_db_con = sqlite3.connect(database_filepath)
neodys_db_cur = neodys_db_con.cursor()

# Get all information from the DB. Since the DB is rather small, this won't cause any issues!
neo_df = pd.read_sql("SELECT * FROM main", neodys_db_con)

# Close the database.
neodys_db_con.close()

In [3]:
# Load SPICE kernels
spiceypy.furnsh("../kernels/spk/de432s.bsp")
spiceypy.furnsh("../kernels/lsk/naif0012.tls")
spiceypy.furnsh("../kernels/pck/gm_de431.tpc")

# Get the G*M value of the Sun
_, gm_sun_pre = spiceypy.bodvcd(bodyid=10, item='GM', maxn=1)
gm_sun = gm_sun_pre[0]

In [4]:
# For our computations we need to convert some values from AU to km and from deg to rad
neo_df.loc[:, "Perihel_km"] = neo_df["Perihel_AU"].apply(lambda x: spiceypy.convrt(x, "AU", "km"))
neo_df.loc[:, "Incl_rad"] = neo_df["Incl_deg"].apply(lambda x: math.radians(x))
neo_df.loc[:, "LongAscNode_rad"] = neo_df["LongAscNode_deg"].apply(lambda x: math.radians(x))
neo_df.loc[:, "ArgP_rad"] = neo_df["ArgP_deg"].apply(lambda x: math.radians(x))
neo_df.loc[:, "MeanAnom_rad"] = neo_df["MeanAnom_deg"].apply(lambda x: math.radians(x))
neo_df.loc[:, "Epoch_JD"] = neo_df["Epoch_MJD"].apply(lambda x: x + 2400000.5)
neo_df.loc[:, "Epoch_et"] = neo_df["Epoch_JD"].apply(lambda x: spiceypy.utc2et(str(x) + " JD"))

In [5]:
# Some simulation parameters (better way: creating a config file for maximum flexibility)

init_time_utc = "2022-01-01T00:00:00"
init_time_et = spiceypy.utc2et(init_time_utc)

# "Circle" around the opposition that is the detection area (in degrees)
opp_range = 15.0

# Minimum detection threshold (in magnitude)
mag_detec = 26.0

# Dataframe that stores the results
detected_neo_df = pd.DataFrame([])

# Simulation steps in seconds
time_step_size = 3600.0

# Observation hour range in hours
obs_range = 8.0

In [6]:
# Simulation loop. For better effieciency this can be done asychronically
for time_step_h in tqdm.tqdm(np.arange(0, obs_range, 1)):

    # Computation time
    init_time_et += time_step_size
    neo_df.loc[:, "et_of_detection"] = init_time_et
    
    # Position vector of the Earth as seen from the Sun
    sun2earth_position_vec = spiceypy.spkgps(targ=399,
                                             et=init_time_et,
                                             ref="ECLIPJ2000",
                                             obs=10)[0]
    earth2sun_position_vec = -1.0 * sun2earth_position_vec

    # Compute the position vector of each NEO as seen from the Sun
    neo_df.loc[:, "sun2neo_position_vec"] = \
        neo_df.apply(lambda x: spiceypy.conics(elts=[x["Perihel_km"],
                                                     x["Ecc_"],
                                                     x["Incl_rad"],
                                                     x["LongAscNode_rad"],
                                                     x["ArgP_rad"],
                                                     x["MeanAnom_rad"],
                                                     x["Epoch_et"],
                                                     gm_sun],
                                             et=init_time_et)[:3],
                     axis=1)

    # To compute the apparent magnitude we need to re-compute the positional vectors and convert it
    # to AU
    neo_df.loc[:, "neo2earth_position_vec"] = \
        neo_df["sun2neo_position_vec"].apply(lambda x: sun2earth_position_vec - x)

    neo_df.loc[:, "neo2sun_position_vec"] = \
        neo_df["sun2neo_position_vec"].apply(lambda x: -1.0 * x)

    neo_df.loc[:, "neo2earth_position_vec_AU"] = \
        neo_df["neo2earth_position_vec"].apply(lambda x:
                                               [spiceypy.convrt(k, "km", "AU") for k in x])
    neo_df.loc[:, "neo2sun_position_vec_AU"] = \
        neo_df["neo2sun_position_vec"].apply(lambda x:
                                             [spiceypy.convrt(k, "km", "AU") for k in x])

    neo_df.loc[:, "earth2neo_position_vec_AU"] = \
        neo_df["neo2earth_position_vec_AU"].apply(lambda x: -1.0 * np.array(x))

    # Compute the apparent magnitude of each NEO
    neo_df.loc[:, "app_mag"] = \
        neo_df.apply(lambda x: photometry.hg_app_mag(abs_mag=x["AbsMag_"],
                                                     vec_obj2obs=x["neo2earth_position_vec_AU"],
                                                     vec_obj2ill=x["neo2sun_position_vec_AU"],
                                                     slope_g=x["SlopeParamG_"]), axis=1)

    # Compute the angular distance between NEO and opposition direction
    neo_df.loc[:, "ang_dist_neo2opp_deg"] = \
        neo_df["earth2neo_position_vec_AU"].apply(lambda x: np.degrees(spiceypy.vsep(x, -1.0*earth2sun_position_vec)))

    # Get the detected NEOs
    detec_rows = neo_df.loc[(neo_df["app_mag"] <= mag_detec) & (neo_df["ang_dist_neo2opp_deg"] <= opp_range), :]

    # ... add them to the detected dataframe, remove them from the simulation
    detected_neo_df = pd.concat([detected_neo_df, detec_rows], ignore_index=True)
    neo_df.drop(detec_rows.index, inplace=True)
    
    # Dataframe empty? Quit
    if len(neo_df) == 0:
        break

100%|█████████████████████████████████████████████████████████████████| 8/8 [00:34<00:00,  4.31s/it]


In [7]:
# Convert the ET detection time to a human-readable format
detected_neo_df.loc[:, "utc_of_detection"] = \
    detected_neo_df["et_of_detection"].apply(lambda x: spiceypy.et2utc(x, "ISOC", 0))

In [8]:
detected_neo_df = detected_neo_df[['Name',
                                   'SemMajAxis_AU',
                                   'Ecc_',
                                   'Incl_deg',
                                   'LongAscNode_deg',
                                   'ArgP_deg',
                                   'AbsMag_',
                                   'SlopeParamG_',
                                   'Aphel_AU',
                                   'Perihel_AU',
                                   'NEOClass',
                                   'utc_of_detection']].copy()

# Store the results in a parquet file
pathlib.Path("results_data/10_simulation").mkdir(parents=True, exist_ok=True)
detected_neo_df.to_parquet(f"results_data/10_simulation/" \
                           + f"UTC{init_time_utc}" \
                           + f"_OppDist{opp_range}" \
                           + f"_MagDetec{mag_detec}" \
                           + f"_StepSize{time_step_size}" \
                           + f"_HourObs{obs_range}.parquet")

In [9]:
detected_neo_df

,Name,SemMajAxis_AU,Ecc_,Incl_deg,LongAscNode_deg,ArgP_deg,AbsMag_,SlopeParamG_,Aphel_AU,Perihel_AU,NEOClass,utc_of_detection
0,3671,2.199030,0.541299,13.529126,82.083031,204.290012,16.49,0.15,3.389363,1.008697,Apollo,2022-01-01T01:00:00
1,6491,2.501206,0.590531,5.947035,301.896475,323.648941,18.53,0.15,3.978244,1.024167,Amor,2022-01-01T01:00:00
2,7358,2.198277,0.502361,4.661553,265.474469,92.207003,14.70,0.15,3.302607,1.093948,Amor,2022-01-01T01:00:00
3,8013,2.199937,0.431445,7.569471,105.562778,146.752731,17.00,0.15,3.149088,1.250786,Amor,2022-01-01T01:00:00
4,19764,2.233160,0.442889,1.325788,281.511048,9.579817,16.09,0.15,3.222201,1.244118,Amor,2022-01-01T01:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
286,2009SK104,2.757622,0.616620,4.153927,328.040842,286.175275,19.41,0.15,4.458026,1.057218,Amor,2022-01-01T05:00:00
287,2016LB9,1.795824,0.482840,7.896406,154.156878,182.278225,21.41,0.15,2.662920,0.928728,Apollo,2022-01-01T05:00:00
288,2014KX99,2.800036,0.589011,13.556852,288.307823,348.524906,18.10,0.15,4.449289,1.150783,Amor,2022-01-01T06:00:00
289,2021PG10,2.777715,0.629997,5.662584,332.323046,51.361468,22.00,0.15,4.527666,1.027764,Amor,2022-01-01T06:00:00
